### Policy Gradient Theorem Derivation
The goal is: 
$$
J(\theta) = \mathbb{E}_{\tau \sim p_\theta}\left[\sum_{t=0}^{\infty} \gamma^t\,r(s_t, a_t)\right]
$$

We can write:
$$
J(\theta) = \int p_\theta(\tau)\left[\sum_{t=0}^{\infty} \gamma^t\,r(s_t,a_t)\right]\,d\tau.
$$

Taking the gradient with respect to $\theta$:
$$
\nabla_\theta J(\theta) = \nabla_\theta \int p_\theta(\tau)\left[\sum_{t=0}^{\infty}\gamma^t\,r(s_t,a_t)\right]d\tau.
$$
Assuming we can interchange the gradient and the integral:
$$
\nabla_\theta J(\theta) = \int \nabla_\theta p_\theta(\tau)\left[\sum_{t=0}^{\infty}\gamma^t\,r(s_t,a_t)\right]d\tau.
$$

Applying the Log-Likelihood trick gives:
$$
\nabla_\theta J(\theta) = \int p_\theta(\tau)\,\nabla_\theta \log p_\theta(\tau) \left[\sum_{t=0}^{\infty}\gamma^t\,r(s_t,a_t)\right]d\tau.
$$
In expectation notation:
$$
\nabla_\theta J(\theta) = \mathbb{E}_{\tau \sim p_\theta}\left[\nabla_\theta \log p_\theta(\tau) \,\sum_{t=0}^{\infty}\gamma^t\,r(s_t,a_t)\right].
$$


Since the trajectory probability factorizes as
$$
p_\theta(\tau) = p(s_0)\prod_{t=0}^{\infty} \left[\pi_\theta(a_t \mid s_t) \,P(s_{t+1} \mid s_t,a_t)\right],
$$
its logarithm is
$$
\log p_\theta(\tau) = \log p(s_0) + \sum_{t=0}^{\infty}\Bigl[\log \pi_\theta(a_t \mid s_t) + \log P(s_{t+1} \mid s_t,a_t)\Bigr].
$$
Only the terms $\log \pi_\theta(a_t \mid s_t)$ depend on $\theta$. Therefore,
$$
\nabla_\theta \log p_\theta(\tau) = \sum_{t=0}^{\infty}\nabla_\theta \log \pi_\theta(a_t \mid s_t).
$$
Substitute back into our gradient expression:
$$
\nabla_\theta J(\theta) = \mathbb{E}_{\tau \sim p_\theta}\left[\left(\sum_{t=0}^{\infty}\nabla_\theta \log \pi_\theta(a_t \mid s_t)\right)\left(\sum_{t=0}^{\infty}\gamma^t\,r(s_t,a_t)\right)\right].
$$

For each time step t, decompose the total return as:
$$
\sum_{m=0}^{\infty}\gamma^m\,r(s_m,a_m)
=\underbrace{\sum_{m=0}^{t-1}\gamma^m\,r(s_m,a_m)}_{\text{past (independent of \(a_t\))}}
+\underbrace{\sum_{m=t}^{\infty}\gamma^m\,r(s_m,a_m)}_{\text{future (dependent on \(a_t\))}}.
$$
Since the past portion is independent of $(a_t)$, its contribution to $(\nabla_\theta \log \pi_\theta(a_t\mid s_t))$ vanishes in expectation. Hence, we have:
$$
\nabla_\theta J(\theta) = \mathbb{E}_{\tau \sim p_\theta}\left[\sum_{t=0}^{\infty}\nabla_\theta \log \pi_\theta(a_t \mid s_t)\left(\sum_{m=t}^{\infty}\gamma^m\,r(s_m,a_m)\right)\right].
$$

Since the Q function is:
$$
Q^\pi(s_t,a_t) = \mathbb{E}\left[\sum_{k=0}^{\infty}\gamma^k\,r(s_{t+k},a_{t+k}) \,\Bigm|\, s_t,a_t\right].
$$
Thus, the partial sum $(\sum_{m=t}^{\infty}\gamma^m\,r(s_m,a_m)$ is an unbiased sample of $(Q^\pi(s_t,a_t))$. Therefore,
$$
\nabla_\theta J(\theta) = \mathbb{E}_{\tau \sim p_\theta}\left[\sum_{t=0}^{\infty}\nabla_\theta \log \pi_\theta(a_t \mid s_t)\,Q^\pi(s_t,a_t)\right].
$$


Each trajectory $(\tau)$ consists of a sequence of state–action pairs $((s_0,a_0), (s_1,a_1), \ldots)$. The sum over time steps is equivalent to taking an expectation with respect to the **discounted occupancy measure** $(\mu_\pi(s,a))$, which represents the (normalized) distribution of state–action pairs visited by $(\pi_\theta)$. Thus, we can write:
$$
\nabla_\theta J(\theta) = \mathbb{E}_{(s,a) \sim \mu_\pi}\left[\nabla_\theta \log \pi_\theta(a \mid s)\,Q^\pi(s,a)\right].
$$

---

**Final Policy Gradient Theorem**

$$
\boxed{
\nabla_\theta J(\theta) = \mathbb{E}_{(s,a) \sim \mu_\pi}\left[\nabla_\theta \log \pi_\theta(a \mid s)\,Q^\pi(s,a)\right].
}
$$
A common variance-reduced version uses the advantage function $A^\pi(s,a)=Q^\pi(s,a)-V^\pi(s)$:
$$
\boxed{
\nabla_\theta J(\theta) = \mathbb{E}_{(s,a) \sim \mu_\pi}\left[\nabla_\theta \log \pi_\theta(a \mid s)\,A^\pi(s,a)\right].
}
$$


### REINFORCE
**Objective:**

The expected return is given by

$$
J(\theta) = \mathbb{E}_{\tau \sim \pi_\theta}\left[\sum_{t=0}^{T-1}\gamma^t\,r(s_t,a_t)\right],
$$

where $ \tau = (s_0,a_0,\dots,s_{T-1},a_{T-1}) $ is a complete episode.

**Policy Gradient Theorem (using Monte Carlo return):**

The gradient is

$$
\nabla_\theta J(\theta) = \mathbb{E}_{\tau \sim \pi_\theta}\left[\sum_{t=0}^{T-1} \nabla_\theta \log \pi_\theta(a_t\mid s_t) \, G_t\right],
$$

with the return

$$
G_t = \sum_{k=t}^{T-1}\gamma^{k-t}\,r(s_k,a_k).
$$

**Update Rule:**

For each episode, update

$$
\theta \leftarrow \theta + \alpha \sum_{t=0}^{T-1} G_t \, \nabla_\theta \log \pi_\theta(a_t\mid s_t).
$$

### Actor Critic (A2C)
**Architecture:**

We have an actor (policy) and a critic (value function). The critic estimates

$$
V^\pi(s) \approx \mathbb{E}\left[G_t \mid s_t = s\right].
$$

**Advantage Estimate:**

A common choice is the one-step temporal difference (TD) error:

$$
A_t = r(s_t,a_t) + \gamma V^\pi(s_{t+1}) - V^\pi(s_t).
$$

**Loss Functions:**

- **Actor Loss:**

$$
L_{\text{actor}}(\theta) = -\mathbb{E}\left[\log \pi_\theta(a_t\mid s_t)\,A_t\right].
$$

- **Critic Loss:**

$$
L_{\text{critic}}(\phi) = \frac{1}{2}\mathbb{E}\left[\left(r(s_t,a_t) + \gamma V_\phi(s_{t+1}) - V_\phi(s_t)\right)^2\right].
$$

**Update Rules:**

- **Actor Update:**

$$
\theta \leftarrow \theta + \alpha\,\nabla_\theta \log \pi_\theta(a_t\mid s_t) \, A_t.
$$

- **Critic Update:**

$$
\phi \leftarrow \phi - \beta\,\nabla_\phi L_{\text{critic}}(\phi).
$$


### Natural Policy Gradient (NPG)
**Key Idea:**

The standard gradient is preconditioned by the inverse Fisher information matrix \( F(\theta) \) to obtain the natural gradient:

$$
\Delta \theta = F(\theta)^{-1} \, \nabla_\theta J(\theta),
$$

where

$$
F(\theta) = \mathbb{E}_{s \sim d^\pi,\, a \sim \pi_\theta}\left[\nabla_\theta \log \pi_\theta(a\mid s) \, \nabla_\theta \log \pi_\theta(a\mid s)^\top\right].
$$

**Update Rule:**

$$
\theta \leftarrow \theta + \alpha \, \Delta \theta = \theta + \alpha \, F(\theta)^{-1} \, \nabla_\theta J(\theta).
$$

### Trust Region Policy Optimization (TRPO)
**Objective:**

TRPO seeks to solve

$$
\max_\theta \; \hat{\mathbb{E}}_{(s,a) \sim \pi_{\theta_{\text{old}}}}\left[\frac{\pi_\theta(a\mid s)}{\pi_{\theta_{\text{old}}}(a\mid s)} \, A^{\theta_{\text{old}}}(s,a)\right]
$$

subject to

$$
\hat{\mathbb{E}}_{s \sim \pi_{\theta_{\text{old}}}}\left[D_{KL}\Bigl(\pi_{\theta_{\text{old}}}(\cdot\mid s) \,\|\, \pi_\theta(\cdot\mid s)\Bigr)\right] \leq \delta.
$$

**Solution Method:**

TRPO uses a conjugate gradient method and line search to find the update $ \Delta\theta $ that approximately solves the constrained problem.

**Update:**

$$
\theta_{\text{new}} = \theta_{\text{old}} + \Delta \theta.
$$

### Proximal Policy Optimization (PPO)